In [1]:
import numpy as np
from scipy.io import loadmat
SA_raw_data = loadmat('sub_a.mat')
SA_signals = SA_raw_data['responses']
SA_label = SA_raw_data['is_stimulate']
SA_data = []
SA_target = []
for i in range(12):
    for j in range(85):
        if SA_label[i, j] == 1:
            SA_data.append(SA_signals[i, :, :, j].reshape(-1, 64))
            SA_target.append(SA_label[i, j])
            SA_data.append(SA_signals[i, :, :, j].reshape(-1, 64))
            SA_target.append(SA_label[i, j])
            SA_data.append(SA_signals[i, :, :, j].reshape(-1, 64))
            SA_target.append(SA_label[i, j])
            SA_data.append(SA_signals[i, :, :, j].reshape(-1, 64))
            SA_target.append(SA_label[i, j])
        SA_data.append(SA_signals[i, :, :, j].reshape(-1, 64))
        SA_target.append(SA_label[i, j])
SA_data = np.array(SA_data)
SA_target = np.array(SA_target)

In [2]:
print(SA_target)

[0 1 1 ... 0 0 0]


In [3]:
SB_raw_data = loadmat('sub_b.mat')
SB_signals = SB_raw_data['responses']
SB_label = SB_raw_data['is_stimulate']
SB_data = []
SB_target = []
for i in range(12):
    for j in range(85):
        if SB_label[i, j] == 1:
            SB_data.append(SB_signals[i, :, :, j].reshape(-1, 64))
            SB_target.append(SB_label[i, j])
            SB_data.append(SB_signals[i, :, :, j].reshape(-1, 64))
            SB_target.append(SB_label[i, j])
            SB_data.append(SB_signals[i, :, :, j].reshape(-1, 64))
            SB_target.append(SB_label[i, j])
            SB_data.append(SB_signals[i, :, :, j].reshape(-1, 64))
            SB_target.append(SB_label[i, j])
        SB_data.append(SB_signals[i, :, :, j].reshape(-1, 64))
        SB_target.append(SB_label[i, j])
SB_data = np.array(SB_data)
SB_target = np.array(SB_target)

In [4]:
def get_num(array):
    num_0=0
    num_1=0
    for i in range(len(array)):
        if array[i]==0:
            num_0+=1
        else:
            num_1+=1
    return num_0,num_1
SA_0,SA_1=get_num(SB_target)
print(SA_0)
print(SA_1)

850
850


In [10]:
y_train_0,y_train_1=get_num(y_train)
print(y_train_0)
print(y_train_1)
y_test_0,y_test_1=get_num(y_test)
print(y_test_0)
print(y_test_1)

1201
1179
499
521


In [5]:
print(SA_data.shape)
print(SA_target.shape)
print(SB_data.shape)
print(SB_target.shape)

(1700, 240, 64)
(1700,)
(1700, 240, 64)
(1700,)


In [6]:
all_data=np.concatenate((SA_data,SB_data),axis=0)
all_target=np.concatenate((SA_target,SB_target),axis=0)

In [9]:
import numpy as np
import _pickle as cp

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers.convolutional import Conv1D
from keras import optimizers
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(all_data, all_target, test_size=0.3, random_state=42)
y_train=np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

In [8]:
NB_SENSOR_CHANNELS = 64 #通道数目
NUM_CLASSES = 2       #类别数
BATCH_SIZE = 64
NUM_FILTERS = 64
FILTER_SIZE = 5
NUM_UNITS_LSTM = 128
STRIDE_SIZE=3 #卷积步长

In [9]:
rmp = optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)

model = Sequential()
model.add(Conv1D(filters=NUM_FILTERS, kernel_size=FILTER_SIZE, strides=STRIDE_SIZE,activation='relu', kernel_initializer='orthogonal',
                 input_shape=(240, NB_SENSOR_CHANNELS), 
                 name='Conv1D_1'))
model.add(Conv1D(filters=NUM_FILTERS, kernel_size=FILTER_SIZE,strides=STRIDE_SIZE, activation='relu', kernel_initializer='orthogonal', 
                 name='Conv1D_2'))
model.add(Conv1D(filters=NUM_FILTERS, kernel_size=FILTER_SIZE,strides=STRIDE_SIZE, activation='relu', kernel_initializer='orthogonal', 
                 name='Conv1D_3'))
#model.add(Conv1D(filters=NUM_FILTERS, kernel_size=FILTER_SIZE, strides=STRIDE_SIZE, activation='relu', kernel_initializer='orthogonal',
#                 name='Conv1D_4'))
model.add(LSTM(NUM_UNITS_LSTM, return_sequences=True, kernel_initializer='orthogonal', 
               name='LSTM_1'))
model.add(LSTM(NUM_UNITS_LSTM, return_sequences=True, kernel_initializer='orthogonal', 
               name='LSTM_2'))
model.add(Flatten(name='Flatten'))
model.add(Dropout(0.5, name='dropout'))
model.add(Dense(NUM_CLASSES, activation='softmax', kernel_initializer='orthogonal', 
                name='Output'))

model.compile(loss='categorical_crossentropy', optimizer=rmp, metrics=['accuracy'])

print(model.summary())

Instructions for updating:
Colocations handled automatically by placer.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Conv1D_1 (Conv1D)            (None, 79, 64)            20544     
_________________________________________________________________
Conv1D_2 (Conv1D)            (None, 25, 64)            20544     
_________________________________________________________________
Conv1D_3 (Conv1D)            (None, 7, 64)             20544     
_________________________________________________________________
LSTM_1 (LSTM)                (None, 7, 128)            98816     
_________________________________________________________________
LSTM_2 (LSTM)                (None, 7, 128)            131584    
_________________________________________________________________
Flatten (Flatten)            (None, 896)               0         
________________________________________________

In [10]:
model.fit(X_train, y_train, epochs=100, batch_size=BATCH_SIZE,verbose=1)

Instructions for updating:
Use tf.cast instead.
Epoch 1/100
2380/2380 [==============================] - 11s 5ms/step - loss: 0.5867 - accuracy: 0.6962
Epoch 2/100
2380/2380 [==============================] - 5s 2ms/step - loss: 0.3113 - accuracy: 0.8689
Epoch 3/100
2380/2380 [==============================] - 5s 2ms/step - loss: 0.1851 - accuracy: 0.9202
Epoch 4/100
2380/2380 [==============================] - 5s 2ms/step - loss: 0.1247 - accuracy: 0.9555
Epoch 5/100
2380/2380 [==============================] - 5s 2ms/step - loss: 0.0975 - accuracy: 0.9626
Epoch 6/100
2380/2380 [==============================] - 6s 2ms/step - loss: 0.0526 - accuracy: 0.9811
Epoch 7/100
2380/2380 [==============================] - 6s 2ms/step - loss: 0.0630 - accuracy: 0.9794
Epoch 8/100
2380/2380 [==============================] - 6s 2ms/step - loss: 0.0550 - accuracy: 0.9811
Epoch 9/100
2380/2380 [==============================] - 5s 2ms/step - loss: 0.0348 - accuracy: 0.9895
Epoch 10/100
2380/2380 [

In [13]:
import time
start =time.clock()
test_pred = np.argmax(model.predict(X_test), axis=1)
test_true = np.argmax(y_test, axis=1)
np.unique(test_pred)
import sklearn.metrics as metrics
print("\tTest accauracy:\t{:.4f} ".format(metrics.accuracy_score(test_true, test_pred)))
end = time.clock()
print('Running time: %s Seconds'%(end-start))

	Test accauracy:	0.9794 
Running time: 0.7415946 Seconds


In [9]:
rmp = optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)

model2 = Sequential()
model2.add(Conv1D(filters=NUM_FILTERS, kernel_size=FILTER_SIZE, strides=STRIDE_SIZE,activation='relu', kernel_initializer='orthogonal',
                 input_shape=(240, NB_SENSOR_CHANNELS), 
                 name='Conv1D_1'))
model2.add(Conv1D(filters=NUM_FILTERS, kernel_size=FILTER_SIZE,strides=STRIDE_SIZE, activation='relu', kernel_initializer='orthogonal', 
                 name='Conv1D_2'))
model2.add(Conv1D(filters=NUM_FILTERS, kernel_size=FILTER_SIZE,strides=STRIDE_SIZE, activation='relu', kernel_initializer='orthogonal', 
                 name='Conv1D_3'))
#model.add(Conv1D(filters=NUM_FILTERS, kernel_size=FILTER_SIZE, strides=STRIDE_SIZE, activation='relu', kernel_initializer='orthogonal',
#                 name='Conv1D_4'))
model2.add(Flatten(name='Flatten1'))
model2.add(Dropout(0.5, name='dropout1'))
model2.add(Dense(128, activation='softmax', kernel_initializer='orthogonal', 
                name='Dense1'))
model2.add(Dropout(0.5, name='dropout2'))
model2.add(Dense(128, activation='softmax', kernel_initializer='orthogonal', 
                name='Dense2'))
model2.add(Dropout(0.5, name='dropout3'))
model2.add(Dense(NUM_CLASSES, activation='softmax', kernel_initializer='orthogonal', 
                name='Output'))

model2.compile(loss='categorical_crossentropy', optimizer=rmp, metrics=['accuracy'])

print(model2.summary())

Instructions for updating:
Colocations handled automatically by placer.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Conv1D_1 (Conv1D)            (None, 79, 64)            20544     
_________________________________________________________________
Conv1D_2 (Conv1D)            (None, 25, 64)            20544     
_________________________________________________________________
Conv1D_3 (Conv1D)            (None, 7, 64)             20544     
_________________________________________________________________
Flatten1 (Flatten)           (None, 448)               0         
_________________________________________________________________
dropout1 (Dropout)           (None, 448)               0         
_________________________________________________________________
Dense1 (Dense)               (None, 128)               57472     
________________________________________________

In [10]:
model2.fit(X_train, y_train, epochs=100, batch_size=BATCH_SIZE,verbose=1)

Instructions for updating:
Use tf.cast instead.
Epoch 1/100
2380/2380 [==============================] - 8s 3ms/step - loss: 0.6930 - accuracy: 0.5076
Epoch 2/100
2380/2380 [==============================] - 3s 1ms/step - loss: 0.6924 - accuracy: 0.5227
Epoch 3/100
2380/2380 [==============================] - 3s 1ms/step - loss: 0.6904 - accuracy: 0.5962
Epoch 4/100
2380/2380 [==============================] - 4s 1ms/step - loss: 0.6884 - accuracy: 0.5992
Epoch 5/100
2380/2380 [==============================] - 3s 1ms/step - loss: 0.6850 - accuracy: 0.6563
Epoch 6/100
2380/2380 [==============================] - 3s 1ms/step - loss: 0.6804 - accuracy: 0.6794
Epoch 7/100
2380/2380 [==============================] - 4s 2ms/step - loss: 0.6760 - accuracy: 0.6735
Epoch 8/100
2380/2380 [==============================] - 3s 1ms/step - loss: 0.6684 - accuracy: 0.6849
Epoch 9/100
2380/2380 [==============================] - 3s 1ms/step - loss: 0.6621 - accuracy: 0.6697
Epoch 10/100
2380/2380 [=

Epoch 76/100
2380/2380 [==============================] - 3s 1ms/step - loss: 0.4832 - accuracy: 0.7231
Epoch 77/100
2380/2380 [==============================] - 4s 1ms/step - loss: 0.4791 - accuracy: 0.7118
Epoch 78/100
2380/2380 [==============================] - 3s 1ms/step - loss: 0.4852 - accuracy: 0.7109
Epoch 79/100
2380/2380 [==============================] - 3s 1ms/step - loss: 0.4588 - accuracy: 0.7239
Epoch 80/100
2380/2380 [==============================] - 3s 1ms/step - loss: 0.4767 - accuracy: 0.7244
Epoch 81/100
2380/2380 [==============================] - 3s 1ms/step - loss: 0.4923 - accuracy: 0.6950
Epoch 82/100
2380/2380 [==============================] - 3s 1ms/step - loss: 0.4712 - accuracy: 0.7168
Epoch 83/100
2380/2380 [==============================] - 3s 1ms/step - loss: 0.4730 - accuracy: 0.7084
Epoch 84/100
2380/2380 [==============================] - 3s 1ms/step - loss: 0.4861 - accuracy: 0.7176
Epoch 85/100
2380/2380 [==============================] - 3s 1ms

In [12]:
test2_pred = np.argmax(model2.predict(X_test), axis=1)
test_true = np.argmax(y_test, axis=1)
np.unique(test2_pred)
import sklearn.metrics as metrics
print("\tTest accauracy:\t{:.4f} ".format(metrics.accuracy_score(test_true, test2_pred)))

	Test accauracy:	0.9461 


In [12]:
print(X_train.shape)
print(X_test.shape)

(2380, 240, 64)
(1020, 240, 64)
